# Phase / AoA Physics Branch + Fusion with PreCNN

This notebook builds the **physics-guided branch** and fuses it with the
PreCNN amplitude branch:

- From phase CSI, build a **clean phase tensor** and extract:
  - either an **AoA feature** $z_{\text{phys}} \in \mathbb{R}^2$
    using the phase difference between two RX chains,
  - or a more generic **phase pattern** feature if AoA is not available.
- Fuse the physics feature $z_{\text{phys}}$ with the PreCNN embedding
  $z_{\text{pre}}$ (from amplitude) using an MLP to obtain a fused
  localization feature $z_{\text{loc}}$.
- On top of $z_{\text{loc}}$, train a **multi-task head** that:
  - classifies presence (empty vs non-empty),
  - regresses 2D coordinates $(x, y)$.

# Imports, Paths, and Load PreCNN Encoder

We reuse the **PreCNN encoder** trained in Step 2.

- Input: preprocessed amplitude window $X_{\text{pre}} \in \mathbb{R}^{T \times C_{\text{pre}}}$.
- Output: embedding vector
  $$
  z_{\text{pre}} \in \mathbb{R}^{\text{DIM\_PRE}},
  $$
  which serves as the amplitude-based representation of the CSI window.

WiFi and physics constants:

- carrier frequency $f_c = 5.18\ \text{GHz}$,
- wavelength
  $$
  \lambda = \frac{c}{f_c},
  $$
- assumed RX antenna spacing $d = D$ (meters),

are used later in the AoA formula.

In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow:", tf.__version__)

# Use the same paths as Step 2
WINDOW_ROOT = "/home/tonyliao/Location_new_aoa_PDF"   # folder containing amp_window_XXXXX.npy
ENC_SAVE_PATH = "/home/tonyliao/Location_new_aoa_PDF/models/prec_encoder.h5"

# Physics branch config
USE_AOA = True   # AoA if ≥2 RX chains; False → phase-pattern features

C   = 3e8        # speed of light (m/s)
FC  = 5.18e9     # adjust to your WiFi channel (Hz)
LAM = C / FC     # wavelength (m)
D   = 0.05       # estimated RX antenna spacing (m) – adjust if known

# Load PreCNN encoder
encoder = tf.keras.models.load_model(ENC_SAVE_PATH)
encoder.summary()



DIM_PRE = encoder.output_shape[-1]
print("PreCNN embedding dim (DIM_PRE):", DIM_PRE)

2026-01-15 13:12:57.691575: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-15 13:12:57.697968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768482777.705372 1273609 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768482777.707578 1273609 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768482777.713252 1273609 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

TensorFlow: 2.19.0


I0000 00:00:1768482778.851130 1273609 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22181 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "PreCNN_Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4, 159)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 4, 64)          │        50,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 2, 128)         │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 2, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ precnn_embedding (Dense)        │ (None, 128)            │        32,896 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,280 (880.00 KB)

 Trainable params: 224,384 (876.50 KB)

 Non-trainable params: 896 (3.50 KB)

PreCNN embedding dim (DIM_PRE): 128


# File Listing and Amp→Phase Mapping

Each CSI window is stored as two files:

- amplitude: `amp_window_XXXXX.npy`,
- phase: `pha_window_XXXXX.npy`.

We:

- list all amplitude files under `WINDOW_ROOT`,
- map each amplitude file to its corresponding phase file by replacing
  the prefix:
  $$
  \text{amp\_window\_XXXXX.npy} \rightarrow \text{pha\_window\_XXXXX.npy}.
  $$

This assumes amplitude and phase files are aligned per window and
stored in the same folder.

In [2]:
def list_amp_files(root):
    return sorted(glob.glob(os.path.join(root, "**", "amp_window_*.npy"), recursive=True))

def amp_to_pha_path(amp_path):
    """
    Map amp_window_XXXXX.npy → pha_window_XXXXX.npy
    Assumes they sit in the same folder.
    """
    return amp_path.replace("amp_window_", "pha_window_")

amp_files = list_amp_files(WINDOW_ROOT)
print("Found amplitude windows:", len(amp_files))

# quick check for phase files existence (first few only)
for p in amp_files[:5]:
    pha_p = amp_to_pha_path(p)
    print("AMP:", os.path.basename(p), "→ PHA:", os.path.basename(pha_p), 
          "exists:", os.path.exists(pha_p))

Found amplitude windows: 404573
AMP: amp_window_00001.npy → PHA: pha_window_00001.npy exists: True
AMP: amp_window_00002.npy → PHA: pha_window_00002.npy exists: True
AMP: amp_window_00003.npy → PHA: pha_window_00003.npy exists: True
AMP: amp_window_00004.npy → PHA: pha_window_00004.npy exists: True
AMP: amp_window_00005.npy → PHA: pha_window_00005.npy exists: True


# Phase Detrending & Cleaning (from phase npy)

Raw phase $ \phi(t, s, a) $ often contains:

- subcarrier-dependent linear trends (hardware imperfections),
- wrapping at multiples of $2\pi$.

For each antenna $a$:

1. For each time index $t$, we fit a straight line over subcarriers:
   $$
   \phi(t, s, a) \approx k_{t,a} \, s + b_{t,a},
   $$
   then subtract it:
   $$
   \tilde{\phi}(t, s, a) =
   \phi(t, s, a) - \bigl(k_{t,a} s + b_{t,a}\bigr).
   $$

2. We then perform **phase unwrapping** over subcarriers $s$ to remove
   $2\pi$ discontinuities, yielding a cleaned phase
   $$
   \phi_{\text{clean}}(t, s, a).
   $$

The function `compute_clean_phase_from_file` returns a list of
$\phi_{\text{clean}}(\cdot, \cdot, a)$ for all RX antennas $a$.

In [3]:
def detrend_phase(phi_ts):
    """
    phi_ts: [T, S] in radians
    Remove linear trend along subcarriers for each time t.
    """
    T, S = phi_ts.shape
    s_idx = np.arange(S)
    out = np.zeros_like(phi_ts, dtype=np.float32)

    for t in range(T):
        k, b = np.polyfit(s_idx, phi_ts[t], deg=1)   # phi ≈ k*s + b
        out[t] = phi_ts[t] - (k * s_idx + b)

    return out


def compute_clean_phase_from_file(pha_path):
    """
    pha_path: pha_window_XXXXX.npy
    file shape: [T, S, A] in radians
    Returns: list phi_clean[a] each shape [T, S] after detrend + unwrap.
    """
    phi_raw = np.load(pha_path).astype(np.float32)   # [T, S, A]
    T, S, A = phi_raw.shape

    phi_clean = []
    for a in range(A):
        phi = phi_raw[:, :, a]                  # [T, S]
        phi_dt = detrend_phase(phi)             # remove linear trend over S
        phi_unw = np.unwrap(phi_dt, axis=1)     # unwrap along subcarriers
        phi_clean.append(phi_unw)

    return phi_clean

# AoA Feature + Phase-pattern Fallback (Step 3 → z_phys)

If at least two RX antennas are available and `USE_AOA = True`,
we compute an **AoA feature** from the phase difference between
antenna 1 and antenna 2.

1. Phase difference between antennas:
   $$
   \Delta \phi(t, s) =
   \phi_{\text{clean}}(t, s, 2) -
   \phi_{\text{clean}}(t, s, 1).
   $$

2. Unwrap and average over all time and subcarriers to obtain a scalar
   $$
   \overline{\Delta \phi} =
   \text{mean}_{t,s} \bigl(\Delta \phi(t, s)\bigr).
   $$

3. Approximate AoA using the standard two-antenna model:
   $$
   \theta \approx
   \arcsin\!\left(
     \frac{\lambda \, \overline{\Delta \phi}}{2 \pi d}
   \right),
   $$
   where $\lambda$ is the wavelength and $d$ is the antenna spacing.

4. Use a **rotation-invariant encoding**:
   $$
   z_{\text{phys}} =
   \bigl[\sin(\theta), \cos(\theta)\bigr] \in \mathbb{R}^2.
   $$

If AoA is not used (or fewer than 2 RX antennas), we fall back to a
**phase pattern**:

- use antenna 1 only,
- compute mean and standard deviation over time per subcarrier:
  $$
  \mu_s = \tfrac{1}{T} \sum_t \phi_{\text{clean}}(t, s, 1),
  \qquad
  \sigma_s = \sqrt{
    \tfrac{1}{T} \sum_t \bigl(\phi_{\text{clean}}(t, s, 1) - \mu_s\bigr)^2
  },
  $$
- downsample over subcarriers and concatenate
  $[\mu_s]$ and $[\sigma_s]$ to obtain a 1D feature vector
  $z_{\text{phys}}$.

In both cases, the output of Step 3 is a physics feature
$$
z_{\text{phys}} \in \mathbb{R}^{\text{DIM\_PHYS}}.
$$

In [4]:
def compute_aoa_feature(phi_clean):
    if len(phi_clean) < 2:
        raise ValueError("AoA mode requires ≥2 RX antennas.")

    phi1 = phi_clean[0]
    phi2 = phi_clean[1]

    dphi = phi2 - phi1
    dphi_unw = np.unwrap(dphi, axis=1)

    dphi_bar = np.mean(dphi_unw)
    dphi_std = np.std(dphi_unw) + 1e-6
    conf = 1.0 / dphi_std  # larger = more stable

    arg = (LAM * dphi_bar) / (2.0 * np.pi * D)
    arg = np.clip(arg, -1.0, 1.0)
    theta = np.arcsin(arg)

    return np.array([np.sin(theta), np.cos(theta), conf], dtype=np.float32)



def compute_phase_pattern(phi_clean, step=4):
    """
    phi_clean: list of [T, S] arrays, uses antenna 0 only.
    step: downsample factor over subcarriers.
    Returns: phase-pattern feature vector (1D).
    """
    phi = phi_clean[0]                 # [T, S]
    mu_s = np.mean(phi, axis=0)        # [S]
    sd_s = np.std(phi,  axis=0)        # [S]

    mu_ds = mu_s[::step]
    sd_ds = sd_s[::step]

    feat = np.concatenate([mu_ds, sd_ds]).astype(np.float32)
    return feat


def step3_phase_branch(pha_path):
    """
    Step 3: from phase window file → physics feature z_phys.
    pha_path: pha_window_XXXXX.npy
    """
    phi_clean = compute_clean_phase_from_file(pha_path)

    if USE_AOA and len(phi_clean) >= 2:
        z_phys = compute_aoa_feature(phi_clean)       # shape (2,)
    else:
        z_phys = compute_phase_pattern(phi_clean)     # shape (2 * S/step,)

    return z_phys

# Fusion MLP

The **FusionMLP** takes the concatenation of:

- $z_{\text{pre}} \in \mathbb{R}^{\text{DIM\_PRE}}$ from PreCNN,
- $z_{\text{phys}} \in \mathbb{R}^{\text{DIM\_PHYS}}$ from the
  physics branch,

and produces a fused localization feature $z_{\text{loc}}$.

Formally:

1. Concatenate features:
   $$
   h_0 = \bigl[z_{\text{pre}}, z_{\text{phys}}\bigr].
   $$

2. Pass through dense layers with ReLU:
   $$
   h_1 = \text{ReLU}(W_1 h_0 + b_1),
   \quad
   h_2 = \text{ReLU}(W_2 h_1 + b_2).
   $$

3. Final fused representation:
   $$
   z_{\text{loc}} =
   \text{ReLU}(W_3 h_2 + b_3)
   \in \mathbb{R}^{128}.
   $$

This $z_{\text{loc}}$ will be used for both presence classification
and coordinate regression.

In [5]:
def build_fusion_mlp(dim_pre, dim_phys, dim_out=128, dropout=0.2):
    inp_pre  = layers.Input(shape=(dim_pre,),  name="z_pre")
    inp_phys = layers.Input(shape=(dim_phys,), name="z_phys")

    # Normalize (stabilizes training)
    pre  = layers.LayerNormalization()(inp_pre)
    phys = layers.LayerNormalization()(inp_phys)

    # Project both to same dimension
    pre_p  = layers.Dense(dim_out, activation="relu")(pre)
    phys_p = layers.Dense(dim_out, activation="relu")(phys)

    # Gate from physics branch (Eq.8: physics unreliable under TTW noise → learn to downweight)
    g = layers.Dense(64, activation="relu")(phys)
    g = layers.Dropout(dropout)(g)
    g = layers.Dense(1, activation="sigmoid", name="phys_gate")(g)      # [B,1]

    # Residual fusion: z = z_pre + g * z_phys
    z = layers.Add()([pre_p, layers.Multiply()([phys_p, g])])

    # Small refinement MLP
    x = layers.Dense(128, activation="relu")(z)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(64, activation="relu")(x)
    z_loc = layers.Dense(dim_out, activation="relu", name="z_loc")(x)

    return models.Model([inp_pre, inp_phys], z_loc, name="FusionMLP")

# Infer DIM_PHYS and Build Fusion Model

To avoid hard-coding the physics feature dimension:

- we run `step3_phase_branch` on one sample phase window,
- read the length of the resulting vector as
  $$
  \text{DIM\_PHYS} = \bigl|z_{\text{phys}}\bigr|,
  $$
- build the FusionMLP with input sizes
  $\text{DIM\_PRE}$ and $\text{DIM\_PHYS}$.

This keeps the fusion architecture consistent with whatever
AoA or phase-pattern feature is actually produced.

In [6]:
# Use the first available window pair to infer DIM_PHYS
if len(amp_files) == 0:
    raise RuntimeError("No amp_window_*.npy found in WINDOW_ROOT")

test_amp = amp_files[0]
test_pha = amp_to_pha_path(test_amp)

z_phys_test = step3_phase_branch(test_pha)
DIM_PHYS = z_phys_test.shape[0]

print("Physics feature dim (DIM_PHYS):", DIM_PHYS)

fusion_model = build_fusion_mlp(DIM_PRE, DIM_PHYS, dim_out=128)
fusion_model.summary()

Physics feature dim (DIM_PHYS): 3


Model: "FusionMLP"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ z_phys (InputLayer) │ (None, 3)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 3)         │          6 │ z_phys[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │        256 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_pre (InputLayer)  │ (None, 128)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 128)       │        256 │ z_pre[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │        512 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ phys_gate (Dense)   │ (None, 1)         │         65 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     16,512 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 128)       │          0 │ dense_1[0][0],    │
│                     │                   │            │ phys_gate[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128)       │          0 │ dense[0][0],      │
│                     │                   │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     16,512 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_loc (Dense)       │ (None, 128)       │      8,320 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 50,695 (198.03 KB)

 Trainable params: 50,695 (198.03 KB)

 Non-trainable params: 0 (0.00 B)

# Helper: Get z_pre from One Amp Window

For each amplitude window file `amp_window_XXXXX.npy`:

1. We convert raw amplitude $|H(t, s, a)|$ into a 2D tensor
   $X_{\text{pre}}(t, c)$ using the Step 2 pipeline:
   - average across antennas,
   - normalize per subcarrier using global $\mu$ and $\sigma$,
   - add temporal mean and std channels.

2. Feed $X_{\text{pre}}$ to the PreCNN encoder to get
   $$
   z_{\text{pre}} = f_{\text{PreCNN}}(X_{\text{pre}}).
   $$

The helper `get_z_pre_from_amp` wraps this process and returns
a single embedding vector $z_{\text{pre}}$.

In [7]:
def get_z_pre_from_amp(amp_path):
    """
    amp_path: amp_window_XXXXX.npy
    Uses Step 2's load_precnn_input + encoder to get z_pre.
    """
    # X_pre: [T, C_pre]
    X_pre = load_precnn_input(amp_path, mu=mu_global, sigma=sigma_global)
    # Keras expects batch dimension
    X_pre_batch = X_pre[np.newaxis, ...]       # [1, T, C_pre]
    z_pre = encoder.predict(X_pre_batch, verbose=0)[0]  # [DIM_PRE]
    return z_pre.astype(np.float32)

# Integrated Step 3 + 4 for One Window

For each pair of files
`amp_window_XXXXX.npy` and `pha_window_XXXXX.npy`:

1. **Amplitude branch (Step 2):**
   $$
   \text{amp window} \Rightarrow
   X_{\text{pre}} \Rightarrow
   z_{\text{pre}}.
   $$

2. **Physics branch (Step 3):**
   $$
   \text{phase window} \Rightarrow
   \phi_{\text{clean}} \Rightarrow
   z_{\text{phys}}.
   $$

3. **Fusion (Step 4):**
   $$
   z_{\text{loc}} =
   f_{\text{fusion}}\bigl(z_{\text{pre}}, z_{\text{phys}}\bigr).
   $$

The function `step3_and_4_for_window` returns
$(z_{\text{pre}}, z_{\text{phys}}, z_{\text{loc}})$ for inspection
and later training.

In [8]:
# === Amp/PreCNN preprocessing helpers (copied/adapted from Step 2) ===
import os
import glob
import numpy as np

# If WINDOW_ROOT is not defined yet, set it here (adjust path if needed)
try:
    WINDOW_ROOT
except NameError:
    WINDOW_ROOT = "/home/tonyliao/Location_new_aoa_PDF"   # folder containing amp_window_XXXXX.npy

def list_amp_files(root):
    return sorted(glob.glob(os.path.join(root, "**", "amp_window_*.npy"), recursive=True))

amp_files = list_amp_files(WINDOW_ROOT)
print("Found amp windows:", len(amp_files))


def compute_mu_sigma(paths, sample_limit=None):
    """
    Compute global per-channel mean and std over avecsi-like amplitude.
    paths: list of amp_window_XXXXX.npy
    """
    sums = None
    sq_sums = None
    count = 0

    for i, p in enumerate(paths):
        if sample_limit and i >= sample_limit:
            break

        raw = np.load(p)                 # [T, S, A]
        avecsi_like = raw.mean(axis=2)   # [T, S]
        amp_ct = avecsi_like.T           # [S, T]

        C, _ = amp_ct.shape
        if sums is None:
            sums = np.zeros((C,), dtype=np.float64)
            sq_sums = np.zeros((C,), dtype=np.float64)

        sums    += amp_ct.mean(axis=1)
        sq_sums += (amp_ct**2).mean(axis=1)
        count   += 1

    mu = sums / count
    sigma = np.sqrt(sq_sums / count - mu**2)
    return mu.astype(np.float32), sigma.astype(np.float32)


print("Computing global mu/sigma over amplitude channels ...")
mu_global, sigma_global = compute_mu_sigma(amp_files)
print("mu_global shape:", mu_global.shape)
print("sigma_global shape:", sigma_global.shape)


# --- PreCNN stats channels (same logic as Step 2) ---
USE_PRECNN_STATS = True  # amp + temporal mean + temporal std

def add_precnn_stats_channels(amp_ct):
    """
    amp_ct: [C_base, T]
    returns: [C_pre, T] with extra mean/std channels.
    """
    C_base, T = amp_ct.shape

    mu = amp_ct.mean(axis=1, keepdims=True)         # [C_base, 1]
    sd = amp_ct.std(axis=1, keepdims=True) + 1e-6   # [C_base, 1]

    mu_rep = np.repeat(mu, T, axis=1)
    sd_rep = np.repeat(sd, T, axis=1)

    return np.concatenate([amp_ct, mu_rep, sd_rep], axis=0).astype(np.float32)


def load_precnn_input(path, mu=None, sigma=None):
    """
    path: amp_window_XXXXX.npy
    raw shape: [T, S, A]

    Output: X_pre [T, C_pre] ready for PreCNN encoder.
    """
    amp = np.load(path).astype(np.float32)   # [T, S, A]

    # Step 1: average over antennas
    avecsi_like = amp.mean(axis=2)           # [T, S]

    # Step 2: convert to channels-first [C, T]
    amp_ct = avecsi_like.T                   # [S, T]

    # Step 3: normalize per-channel
    if mu is not None and sigma is not None:
        amp_ct = (amp_ct - mu[:, None]) / (sigma[:, None] + 1e-6)

    # Step 4: add PreCNN stat channels
    if USE_PRECNN_STATS:
        amp_ct = add_precnn_stats_channels(amp_ct)   # [C_pre, T]

    # Step 5: final format for Keras: [T, C_pre]
    X_pre = np.transpose(amp_ct, (1, 0))
    return X_pre


# --- Helper: get z_pre from one amp window using loaded encoder ---
def get_z_pre_from_amp(amp_path):
    """
    amp_path: amp_window_XXXXX.npy
    Uses load_precnn_input + encoder (from Step 2) to get z_pre.
    Requires:
      - encoder already loaded (tf.keras.models.load_model)
      - mu_global, sigma_global computed above
    """
    # X_pre: [T, C_pre]
    X_pre = load_precnn_input(amp_path, mu=mu_global, sigma=sigma_global)
    X_pre_batch = X_pre[np.newaxis, ...]  # [1, T, C_pre]
    z_pre = encoder.predict(X_pre_batch, verbose=0)[0]  # [DIM_PRE]
    return z_pre.astype(np.float32)


Found amp windows: 404573
Computing global mu/sigma over amplitude channels ...
mu_global shape: (53,)
sigma_global shape: (53,)


In [9]:
def amp_adj_corr_feature(amp_path):
    amp = np.load(amp_path).astype(np.float32)      # [T,S,A]
    amp_ct = amp.mean(axis=2).T                     # [S,T]
    amp_ct = amp_ct - np.median(amp_ct, axis=1, keepdims=True)

    X = (amp_ct - amp_ct.mean(axis=1, keepdims=True)) / (amp_ct.std(axis=1, keepdims=True)+1e-6)
    adj_corr = float(np.mean(np.mean(X[:-1] * X[1:], axis=1)))     # Eq(7): cross-subcarrier structure
    dyn_energy = float(np.mean(amp_ct**2))                          # Eq(5)-(7): dynamic strength
    return np.array([adj_corr, dyn_energy], dtype=np.float32)

def step3_and_4_for_window(amp_path):
    """
    amp_path: amp_window_XXXXX.npy
    Returns:
        z_pre  : PreCNN feature           (DIM_PRE,)
        z_phys : physics feature          (DIM_PHYS,)
        z_loc  : fused localization feat  (dim_out,)
    """
    pha_path = amp_to_pha_path(amp_path)

    # Step 2 → PreCNN feature
    z_pre = get_z_pre_from_amp(amp_path)          # (DIM_PRE,)
    
    # Step 3 → physics feature
    z_phys = step3_phase_branch(pha_path)         # (DIM_PHYS,)
    z_ampfeat = amp_adj_corr_feature(amp_path)
    z_phys = np.concatenate([z_phys, z_ampfeat], axis=0)
    # Step 4 → fusion
    z_pre_in  = z_pre.reshape(1, -1)
    z_phys_in = z_phys.reshape(1, -1)
    z_loc = fusion_model([z_pre_in, z_phys_in]).numpy()[0]

    return z_pre, z_phys, z_loc

In [10]:
# ============================================================
# Rebuild FusionMLP with correct DIM_PHYS (includes +2 amp feats)
# ============================================================
import os
import numpy as np

def step3_and_4_extract_only(amp_path):
    """Return (z_pre, z_phys) WITHOUT calling fusion_model."""
    pha_path = amp_to_pha_path(amp_path)

    z_pre = get_z_pre_from_amp(amp_path).astype(np.float32)

    z_phys = step3_phase_branch(pha_path).astype(np.float32)   # e.g., AoA: (3,) = [sin,cos,conf]
    z_ampfeat = amp_adj_corr_feature(amp_path).astype(np.float32)  # (2,) = [adj_corr, dyn_energy]

    z_phys = np.concatenate([z_phys, z_ampfeat], axis=0).astype(np.float32)  # => (5,)
    return z_pre, z_phys

# pick one valid amp that has pha
def find_valid_amp(amp_list):
    for p in amp_list:
        if os.path.exists(amp_to_pha_path(p)):
            return p
    raise RuntimeError("No (amp, pha) pair found. Check WINDOW_ROOT and naming.")

test_amp = find_valid_amp(amp_files)
z_pre0, z_phys0 = step3_and_4_extract_only(test_amp)

DIM_PRE  = int(z_pre0.shape[0])     # should be 128
DIM_PHYS = int(z_phys0.shape[0])    # should be 5 now
print("Detected DIM_PRE =", DIM_PRE, "DIM_PHYS =", DIM_PHYS)

# IMPORTANT: rebuild fusion_model with new DIM_PHYS
fusion_model = build_fusion_mlp(DIM_PRE, DIM_PHYS, dim_out=128)
fusion_model.summary()

# helper that uses the rebuilt fusion_model
def step3_and_4_for_window(amp_path):
    z_pre, z_phys = step3_and_4_extract_only(amp_path)

    exp_phys = fusion_model.input_shape[1][1]
    if z_phys.shape[0] != exp_phys:
        raise ValueError(f"z_phys dim mismatch: got {z_phys.shape[0]} expected {exp_phys}")

    z_loc = fusion_model([z_pre.reshape(1, -1), z_phys.reshape(1, -1)]).numpy()[0]
    return z_pre, z_phys, z_loc
# ============================================================
# Build feature arrays with consistent z_phys (DIM_PHYS = 5)
# ============================================================
Z_PRE_list  = []
Z_PHYS_list = []

for amp_path in amp_files:
    amp_path = str(amp_path)
    pha_path = amp_to_pha_path(amp_path)
    if not os.path.exists(pha_path):
        continue

    # use the same extractor as inference
    z_pre, z_phys = step3_and_4_extract_only(amp_path)

    Z_PRE_list.append(z_pre)
    Z_PHYS_list.append(z_phys)

Z_PRE  = np.stack(Z_PRE_list,  axis=0)
Z_PHYS = np.stack(Z_PHYS_list, axis=0)

print("Z_PRE shape :", Z_PRE.shape)
print("Z_PHYS shape:", Z_PHYS.shape)   # should be (N, 5)

I0000 00:00:1768482795.547887 1273754 service.cc:152] XLA service 0x7f62cc015e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768482795.547899 1273754 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2026-01-15 13:13:15.552047: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1768482795.572296 1273754 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1768482795.745080 1273754 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Detected DIM_PRE = 128 DIM_PHYS = 5


Model: "FusionMLP"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ z_phys (InputLayer) │ (None, 5)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5)         │         10 │ z_phys[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │        384 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_pre (InputLayer)  │ (None, 128)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128)       │        256 │ z_pre[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │        768 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ phys_gate (Dense)   │ (None, 1)         │         65 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     16,512 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 128)       │          0 │ dense_6[0][0],    │
│ (Multiply)          │                   │            │ phys_gate[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 128)       │          0 │ dense_5[0][0],    │
│                     │                   │            │ multiply_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 128)       │     16,512 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 64)        │      8,256 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_loc (Dense)       │ (None, 128)       │      8,320 │ dense_9[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 51,083 (199.54 KB)

 Trainable params: 51,083 (199.54 KB)

 Non-trainable params: 0 (0.00 B)

Z_PRE shape : (404426, 128)
Z_PHYS shape: (404426, 5)


# Quick Test on First Few Windows

We run `step3_and_4_for_window` on a few sample windows to verify:

- `z_pre` has shape $(\text{DIM\_PRE},)$,
- `z_phys` has shape $(\text{DIM\_PHYS},)$,
- `z_loc` has the desired fusion dimension (e.g. $128$).

This checks that amplitude preprocessing, phase branch, and fusion
are all consistent and compatible before training the multi-task model.

In [11]:
for amp_p in amp_files[:5]:
    print("\n=== Window:", os.path.basename(amp_p), "===")
    try:
        z_pre, z_phys, z_loc = step3_and_4_for_window(amp_p)
        DIM_PRE  = z_pre.shape[0]
        DIM_PHYS = z_phys.shape[0]
    except FileNotFoundError:
        print("Missing pha_window for this amp_window, skip.")
        continue

    print("z_pre  shape:", z_pre.shape)
    print("z_phys shape:", z_phys.shape)
    print("z_loc  shape:", z_loc.shape)
    print("z_loc  (first 8):", z_loc[:8])


=== Window: amp_window_00001.npy ===
z_pre  shape: (128,)
z_phys shape: (5,)
z_loc  shape: (128,)
z_loc  (first 8): [0.         0.34705353 0.         0.08194862 0.         0.09576242
 0.         0.26619732]

=== Window: amp_window_00002.npy ===
z_pre  shape: (128,)
z_phys shape: (5,)
z_loc  shape: (128,)
z_loc  (first 8): [0.         0.39164    0.         0.1556558  0.         0.12534852
 0.         0.25287256]

=== Window: amp_window_00003.npy ===
z_pre  shape: (128,)
z_phys shape: (5,)
z_loc  shape: (128,)
z_loc  (first 8): [0.         0.3673228  0.         0.1481039  0.         0.15980183
 0.         0.23132464]

=== Window: amp_window_00004.npy ===
z_pre  shape: (128,)
z_phys shape: (5,)
z_loc  shape: (128,)
z_loc  (first 8): [0.         0.34259868 0.         0.19733472 0.         0.1712453
 0.         0.25502792]

=== Window: amp_window_00005.npy ===
z_pre  shape: (128,)
z_phys shape: (5,)
z_loc  shape: (128,)
z_loc  (first 8): [0.         0.20353001 0.         0.25676847 0.     

# Config

We configure:

- paths for saving the full multi-task model,
- training hyperparameters:
  - batch size,
  - number of epochs,
  - learning rate,
  - validation split,
- a simple **presence label** from file names:

  - if the path contains `"EMPTY"` we set $y_{\text{cls}} = 0$,
  - otherwise $y_{\text{cls}} = 1$ (non-empty / stationary).

These labels are used for the presence head of the multi-task model.

In [12]:
from tensorflow.keras import optimizers, losses, callbacks

# where to save the full multi-task model
FULL_MODEL_PATH = "/home/tonyliao/Location_new_aoa_PDF/models/loc_full_model.h5"

# training config (can adjust)
BATCH_SIZE = 64
EPOCHS = 40
LR = 1e-3
VAL_SPLIT = 0.2
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)

# simple presence labels from path
def infer_label(path):
    up = path.upper()
    if "EMPTY" in up:
        return 0
    return 1  # NON-EMPTY / STATIONARY


# Build Multi-task Model on Top of Fusion

We define a **multi-task network** on top of the fused feature
$z_{\text{loc}}$:

- Input: $(z_{\text{pre}}, z_{\text{phys}})$,
- Fusion: $z_{\text{loc}} = f_{\text{fusion}}(z_{\text{pre}}, z_{\text{phys}})$,
- Outputs:
  - presence head:
    $$
    \hat{y}_{\text{cls}} =
    \text{softmax}(W_{\text{cls}} z_{\text{loc}} + b_{\text{cls}}),
    $$
  - coordinate head:
    $$
    \hat{\mathbf{p}} =
    W_{\text{loc}} z_{\text{loc}} + b_{\text{loc}}
    \in \mathbb{R}^2.
    $$

So the model jointly predicts:

- a probability over $\{\text{EMPTY}, \text{NONEMPTY}\}$,
- a 2D position estimate $(\hat{x}, \hat{y})$.

In [13]:
def build_full_multitask_model(fusion_model, dim_pre, dim_phys, num_classes=2):
    """
    Inputs:
      z_pre  : [DIM_PRE]
      z_phys : [DIM_PHYS]
    Outputs:
      presence: softmax over {EMPTY, NONEMPTY}
      coords  : (x, y) regression
    """
    inp_pre  = layers.Input(shape=(dim_pre,),  name="z_pre_input")
    inp_phys = layers.Input(shape=(dim_phys,), name="z_phys_input")

    # reuse fusion_model to get z_loc
    z_loc = fusion_model([inp_pre, inp_phys])

    presence = layers.Dense(num_classes, activation="softmax", name="presence")(z_loc)
    coords   = layers.Dense(2, activation="linear",       name="coords")(z_loc)

    model = models.Model([inp_pre, inp_phys], [presence, coords], name="LocPresenceModel")
    return model


full_model = build_full_multitask_model(fusion_model, DIM_PRE, DIM_PHYS, num_classes=2)
full_model.summary()

Model: "LocPresenceModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ z_pre_input         │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_phys_input        │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FusionMLP           │ (None, 128)       │     51,083 │ z_pre_input[0][0… │
│ (Functional)        │                   │            │ z_phys_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ presence (Dense)    │ (None, 2)         │        258 │ FusionMLP[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ coords (Dense)      │ (None, 2)         │        258 │ FusionMLP[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 51,599 (201.56 KB)

 Trainable params: 51,599 (201.56 KB)

 Non-trainable params: 0 (0.00 B)

# Build Feature & Label Arrays (z_pre, z_phys, y_cls, y_loc)

We now construct the training dataset at the **feature level**:

1. Use `amp_list_loc.npy` to get the exact list of amplitude windows
   that have localization labels.

2. For each path in this list:
   - compute $z_{\text{pre}}$ from amplitude,
   - compute $z_{\text{phys}}$ from phase,
   - assign presence label $y_{\text{cls}}$ (here set to $1$,
     because all these windows correspond to "person present").

3. Load coordinates from `coords.npy` to obtain ground-truth
   $$\mathbf{p} = (x, y).$$

We end up with arrays:

- $Z_{\text{PRE}} \in \mathbb{R}^{N \times \text{DIM\_PRE}}$,
- $Z_{\text{PHYS}} \in \mathbb{R}^{N \times \text{DIM\_PHYS}}$,
- $Y_{\text{cls}} \in \{0,1\}^N$,
- $Y_{\text{loc}} \in \mathbb{R}^{N \times 2}$.

In [14]:
import os
import numpy as np

# === Use the filtered list of localization windows only ===
AMP_LIST_PATH    = "/home/tonyliao/Location_new_aoa_PDF/amp_list_loc.npy"
COORDS_NPY_PATH  = "/home/tonyliao/Location_new_aoa_PDF/coords.npy"

# Load the exact amp file list used when building coords.npy
amp_files = np.load(AMP_LIST_PATH, allow_pickle=True)
print("amp_files loaded:", amp_files.shape)

Z_PRE_list  = []
Z_PHYS_list = []
Y_cls_list  = []

for amp_path in amp_files:
    amp_path = str(amp_path)
    pha_path = amp_to_pha_path(amp_path)
    if not os.path.exists(pha_path):
        print("WARN: missing phase for", amp_path, "— skip")
        continue

    # ✅ Use the SAME extractor as inference (z_phys will be 5-D now)
    z_pre, z_phys = step3_and_4_extract_only(amp_path)

    Z_PRE_list.append(z_pre)
    Z_PHYS_list.append(z_phys)

Z_PRE  = np.stack(Z_PRE_list,  axis=0)   # [N, DIM_PRE]
Z_PHYS = np.stack(Z_PHYS_list, axis=0)   # [N, 5]
Y_cls  = np.ones((Z_PRE.shape[0],), dtype=np.int32)

print("Z_PRE shape :", Z_PRE.shape)
print("Z_PHYS shape:", Z_PHYS.shape)  # should be (N, 5)
print("Y_cls shape :", Y_cls.shape)

# === Load coordinates, must match the same N ===
if os.path.exists(COORDS_NPY_PATH):
    Y_loc = np.load(COORDS_NPY_PATH).astype(np.float32)
    if Y_loc.shape[0] != Z_PRE.shape[0] or Y_loc.shape[1] != 2:
        raise ValueError(
            f"coords.npy shape {Y_loc.shape} mismatches features {Z_PRE.shape[0]} x 2"
        )
    print("Loaded coordinates from", COORDS_NPY_PATH)
else:
    print("WARNING: coords.npy not found, using zeros for Y_loc (please replace with real coordinates).")
    Y_loc = np.zeros((Z_PRE.shape[0], 2), dtype=np.float32)


amp_files loaded: (166149,)
Z_PRE shape : (166149, 128)
Z_PHYS shape: (166149, 5)
Y_cls shape : (166149,)
Loaded coordinates from /home/tonyliao/Location_new_aoa_PDF/coords.npy


# Train/Val Split (on feature level)

We shuffle indices and split on the **feature arrays**:

- training set: about $(1 - \text{VAL\_SPLIT})$ of the data,
- validation set: about $\text{VAL\_SPLIT}$ of the data.

The split is applied consistently to:

- $Z_{\text{PRE}}$,
- $Z_{\text{PHYS}}$,
- $Y_{\text{cls}}$,
- $Y_{\text{loc}}$,

so that each sample keeps its paired features and labels.

In [15]:
N = Z_PRE.shape[0]
idx = np.arange(N)
np.random.shuffle(idx)

split = int(N * (1 - VAL_SPLIT))
train_idx = idx[:split]
val_idx   = idx[split:]

Z_PRE_train,  Z_PRE_val  = Z_PRE[train_idx],  Z_PRE[val_idx]
Z_PHYS_train, Z_PHYS_val = Z_PHYS[train_idx], Z_PHYS[val_idx]
Y_cls_train,  Y_cls_val  = Y_cls[train_idx],  Y_cls[val_idx]
Y_loc_train,  Y_loc_val  = Y_loc[train_idx],  Y_loc[val_idx]

print("Train N:", Z_PRE_train.shape[0], "Val N:", Z_PRE_val.shape[0])

Train N: 132919 Val N: 33230


# Cell 5 — Compile & Train Full Model, Save It

We train the full multi-task model with a combined loss:

- classification loss for presence
  $$
  \mathcal{L}_{\text{cls}}
  = - \log p_{y_{\text{cls}}},
  $$
  where $p_{y_{\text{cls}}}$ is the predicted probability of the
  true class,

- regression loss for coordinates
  $$
  \mathcal{L}_{\text{loc}} =
  \left\| \hat{\mathbf{p}} - \mathbf{p} \right\|_2^2
  = (\hat{x} - x)^2 + (\hat{y} - y)^2.
  $$

The total loss is

$$
\mathcal{L}_{\text{total}} =
\mathcal{L}_{\text{cls}} + \mathcal{L}_{\text{loc}}.
$$

We use Adam with learning rate $10^{-3}$, early stopping, and we monitor
`val_presence_accuracy`. The best model is saved to
`loc_full_model.h5`.

In [16]:
from tensorflow.keras import optimizers, losses, callbacks

def euclid_mean(y_true, y_pred):
    return tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(y_true - y_pred), axis=-1)))

LR = 1e-3

full_model.compile(
    optimizer=optimizers.Adam(LR),
    loss={
        "presence": losses.SparseCategoricalCrossentropy(),
        "coords":   losses.MeanSquaredError(),
    },
    loss_weights={
        "presence": 0.0,   # <-- IMPORTANT
        "coords":   1.0,
    },
    metrics={
        "presence": ["accuracy"],   # optional; meaningless here
        "coords":   ["mse"],
    },
)

ckpt = callbacks.ModelCheckpoint(
    FULL_MODEL_PATH,
    save_best_only=True,
    monitor="val_coords_loss",
    mode="min",
    verbose=1,
)

es = callbacks.EarlyStopping(
    monitor="val_coords_loss",
    mode="min",
    patience=10,
    restore_best_weights=True,
    verbose=1,
)


history = full_model.fit(
    x=[Z_PRE_train, Z_PHYS_train],
    y={"presence": Y_cls_train, "coords": Y_loc_train},
    validation_data=(
        [Z_PRE_val, Z_PHYS_val],
        {"presence": Y_cls_val, "coords": Y_loc_val},
    ),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[ckpt, es],
    verbose=2,
)

print("Full multi-task model saved to:", FULL_MODEL_PATH)

np.save("/home/tonyliao/Location_new_aoa_PDF/Z_PRE.npy",  Z_PRE)
np.save("/home/tonyliao/Location_new_aoa_PDF/Z_PHYS.npy", Z_PHYS)
np.save("/home/tonyliao/Location_new_aoa_PDF/Y_cls.npy",  Y_cls)
np.save("/home/tonyliao/Location_new_aoa_PDF/Y_loc.npy",  Y_loc)

Epoch 1/40


2026-01-15 18:45:35.416706: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5', 12 bytes spill stores, 12 bytes spill loads

2026-01-15 18:45:37.167503: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 8 bytes spill stores, 8 bytes spill loads

2026-01-15 18:45:37.171697: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 12 bytes spill stores, 12 bytes spill loads

2026-01-15 18:45:37.231041: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 112 bytes spill stores, 112 bytes spill loads

2026-01-15 18:45:38.456886: I external/local_x


Epoch 1: val_coords_loss improved from inf to 1.44844, saving model to /home/tonyliao/Location_new_aoa_PDF/models/loc_full_model.h5


2077/2077 - 12s - 6ms/step - coords_loss: 1.6711 - coords_mse: 1.6711 - loss: 1.6711 - presence_accuracy: 0.1537 - presence_loss: 0.8844 - val_coords_loss: 1.4484 - val_coords_mse: 1.4492 - val_loss: 1.4492 - val_presence_accuracy: 0.0997 - val_presence_loss: 0.8261
Epoch 2/40

Epoch 2: val_coords_loss improved from 1.44844 to 1.34633, saving model to /home/tonyliao/Location_new_aoa_PDF/models/loc_full_model.h5


2077/2077 - 4s - 2ms/step - coords_loss: 1.4484 - coords_mse: 1.4485 - loss: 1.4485 - presence_accuracy: 0.2107 - presence_loss: 0.7789 - val_coords_loss: 1.3463 - val_coords_mse: 1.3474 - val_loss: 1.3474 - val_presence_accuracy: 0.2413 - val_presence_loss: 0.7610
Epoch 3/40

Epoch 3: val_coords_loss improved from 1.34633 to 1.32381, saving model to /home/tonyliao/Location_new_aoa_PDF/models/loc_full_model.h5


...

2077/2077 - 4s - 2ms/step - coords_loss: 1.1988 - coords_mse: 1.1988 - loss: 1.1988 - presence_accuracy: 0.1411 - presence_loss: 0.7915 - val_coords_loss: 1.1940 - val_coords_mse: 1.1956 - val_loss: 1.1956 - val_presence_accuracy: 0.1312 - val_presence_loss: 0.7873
Epoch 21/40

Epoch 21: val_coords_loss did not improve from 1.19400
2077/2077 - 5s - 2ms/step - coords_loss: 1.1919 - coords_mse: 1.1919 - loss: 1.1919 - presence_accuracy: 0.1407 - presence_loss: 0.7998 - val_coords_loss: 1.1977 - val_coords_mse: 1.1993 - val_loss: 1.1993 - val_presence_accuracy: 0.1184 - val_presence_loss: 0.8123
Epoch 22/40

Epoch 22: val_coords_loss did not improve from 1.19400
2077/2077 - 4s - 2ms/step - coords_loss: 1.1881 - coords_mse: 1.1881 - loss: 1.1881 - presence_accuracy: 0.1594 - presence_loss: 0.8125 - val_coords_loss: 1.1955 - val_coords_mse: 1.1971 - val_loss: 1.1971 - val_presence_accuracy: 0.1508 - val_presence_loss: 0.8240
Epoch 23/40

Epoch 23: val_coords_loss did not improve from 1.1940

2077/2077 - 5s - 2ms/step - coords_loss: 1.1817 - coords_mse: 1.1817 - loss: 1.1817 - presence_accuracy: 0.1606 - presence_loss: 0.8361 - val_coords_loss: 1.1927 - val_coords_mse: 1.1943 - val_loss: 1.1943 - val_presence_accuracy: 0.1826 - val_presence_loss: 0.8255
Epoch 25/40

Epoch 25: val_coords_loss improved from 1.19267 to 1.19245, saving model to /home/tonyliao/Location_new_aoa_PDF/models/loc_full_model.h5


2077/2077 - 4s - 2ms/step - coords_loss: 1.1809 - coords_mse: 1.1809 - loss: 1.1809 - presence_accuracy: 0.1688 - presence_loss: 0.8286 - val_coords_loss: 1.1925 - val_coords_mse: 1.1940 - val_loss: 1.1940 - val_presence_accuracy: 0.1578 - val_presence_loss: 0.8278
Epoch 26/40

Epoch 26: val_coords_loss improved from 1.19245 to 1.19197, saving model to /home/tonyliao/Location_new_aoa_PDF/models/loc_full_model.h5


2077/2077 - 4s - 2ms/step - coords_loss: 1.1773 - coords_mse: 1.1774 - loss: 1.1774 - presence_accuracy: 0.1616 - presence_loss: 0.8320 - val_coords_loss: 1.1920 - val_coords_mse: 1.1935 - val_loss: 1.1935 - val_presence_accuracy: 0.1254 - val_presence_loss: 0.8342
Epoch 27/40

Epoch 27: val_coords_loss did not improve from 1.19197
2077/2077 - 5s - 2ms/step - coords_loss: 1.1697 - coords_mse: 1.1698 - loss: 1.1698 - presence_accuracy: 0.1385 - presence_loss: 0.8430 - val_coords_loss: 1.2120 - val_coords_mse: 1.2135 - val_loss: 1.2135 - val_presence_accuracy: 0.1007 - val_presence_loss: 0.8498
Epoch 28/40

Epoch 28: val_coords_loss did not improve from 1.19197
2077/2077 - 4s - 2ms/step - coords_loss: 1.1723 - coords_mse: 1.1723 - loss: 1.1723 - presence_accuracy: 0.1303 - presence_loss: 0.8429 - val_coords_loss: 1.2069 - val_coords_mse: 1.2085 - val_loss: 1.2085 - val_presence_accuracy: 0.1149 - val_presence_loss: 0.8358
Epoch 29/40

Epoch 29: val_coords_loss did not improve from 1.1919

2077/2077 - 5s - 2ms/step - coords_loss: 1.1498 - coords_mse: 1.1498 - loss: 1.1498 - presence_accuracy: 0.1065 - presence_loss: 0.9114 - val_coords_loss: 1.1893 - val_coords_mse: 1.1910 - val_loss: 1.1910 - val_presence_accuracy: 0.1041 - val_presence_loss: 0.9003
Epoch 36/40

Epoch 36: val_coords_loss did not improve from 1.18935
2077/2077 - 6s - 3ms/step - coords_loss: 1.1475 - coords_mse: 1.1475 - loss: 1.1475 - presence_accuracy: 0.1070 - presence_loss: 0.9167 - val_coords_loss: 1.1977 - val_coords_mse: 1.1994 - val_loss: 1.1994 - val_presence_accuracy: 0.1107 - val_presence_loss: 0.9014
Epoch 37/40

Epoch 37: val_coords_loss did not improve from 1.18935
2077/2077 - 4s - 2ms/step - coords_loss: 1.1441 - coords_mse: 1.1441 - loss: 1.1441 - presence_accuracy: 0.1182 - presence_loss: 0.9022 - val_coords_loss: 1.2004 - val_coords_mse: 1.2021 - val_loss: 1.2021 - val_presence_accuracy: 0.1225 - val_presence_loss: 0.8775
Epoch 38/40

Epoch 38: val_coords_loss did not improve from 1.1893

# Evaluation and Curves

We evaluate the trained model on the validation split:

- report overall loss and per-head metrics,
- inspect the confusion matrix for presence,
- plot accuracy and loss curves over epochs,
- visualize regression quality with a 2D histogram of true vs predicted
  coordinates.

This gives a quick view of both detection and localization performance
before fine-tuning (Step 6).

# Quick Test

As a final sanity check, we:

- pick one example window,
- compute $z_{\text{pre}}$ and $z_{\text{phys}}$,
- run the full model once to obtain:

  - presence probabilities $\hat{y}_{\text{cls}}$,
  - predicted coordinates $\hat{\mathbf{p}} = (\hat{x}, \hat{y})$.

This confirms that the full pipeline
(amplitude → PreCNN, phase → physics branch, fusion, multi-task head)
runs end-to-end on a single CSI window.

In [ ]:
test_amp = amp_files[0]
test_pha = amp_to_pha_path(test_amp)

# use the extractor that returns the full z_phys (includes amp features) to match DIM_PHYS=5
z_pre, z_phys = step3_and_4_extract_only(test_amp)

pred_presence, pred_coords = full_model.predict(
    [z_pre[np.newaxis, :], z_phys[np.newaxis, :]],
    verbose=0
)

print("Presence logits:", pred_presence[0])
print("Pred class:", np.argmax(pred_presence[0]))
print("Pred coords (x,y):", pred_coords[0])

Presence logits: [0.61919594 0.38080403]
Pred class: 0
Pred coords (x,y): [3.542407  1.9107603]


# Reset Kernel

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()
#Restart the kernel to free memory
import IPython
app = IPython.get_ipython()
app.kernel.do_shutdown(True)  # True = restart, False = shutdown

{'status': 'ok', 'restart': True}

: 